# **"Tangan Hijau, Teknologi Cerdas: Mengaplikasikan Machine Learning untuk Pertumbuhan dan Nutrisi Tanaman yang Lebih Efisien"**

> **NURRAHMAWATI**

> Talent Fair Challenge: Aria Dataset

Linkedin: https://www.linkedin.com/in/nurrahmawatii/<br>
Github: http://github.com/nurrahmawatii<br>
Gmail: nurrahmawati682@gmail.com<br>
Phone: +6289647118538

## **I. PENDAHULUAN**

Data ini berisi informasi mengenai variabel yang dapat digunakan untuk memprediksi nutrisi tanaman. Variabel ini termasuk variabel V1-V8, yang mengandung informasi terkait nutrisi tanaman, serta variabel sample_type, yang mengandung informasi tentang jenis sampel dan laboratorium tempat sampel diperoleh.

Proyek ini bertujuan untuk meningkatkan efisiensi dalam pertumbuhan dan nutrisi tanaman dengan menerapkan teknologi cerdas. Data yang digunakan meliputi informasi tentang nutrisi tanaman dan jenis sampel yang berbeda. Dalam proyek ini, machine learning akan digunakan untuk membuat model prediksi yang dapat memprediksi nutrisi tanaman berdasarkan variabel yang relevan.

Pada tahap analisis, kita akan menganalisis perbedaan pengaruh jenis sampel yang berbeda terhadap kualitas tanaman, sehingga kita dapat menentukan apakah jenis sampel mempengaruhi pertumbuhan dan nutrisi tanaman secara signifikan.

Dalam proyek ini, teknologi cerdas (represented by the term "tangan hijau, teknologi cerdas") dan machine learning digunakan untuk memperbaiki pertumbuhan dan nutrisi tanaman dengan cara yang lebih efisien. Diharapkan bahwa proyek ini dapat memberikan manfaat bagi para petani untuk meningkatkan produksi pertanian mereka dan juga bagi lingkungan dengan mengurangi penggunaan pupuk yang tidak perlu.

## **II. IMPORT PUSTAKA DAN MEMUAT DATA**

In [2]:
!pip install auto-sklearn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Split Dataset and Standarize the Datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, OrdinalEncoder

from feature_engine.outliers import Winsorizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as imbpipe
from imblearn.over_sampling import SMOTE

import autosklearn.regression
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

ModuleNotFoundError: No module named 'autosklearn'

Tahap awal yang harus saya lakukan adalah memuat data yang diperoleh dari Google Drive ke dalam notebook saya, sehingga saya tidak perlu lagi mengunggah data secara manual ke dalam notebook.

In [ ]:
# Connecting google colab runtime with google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
data_ori = pd.read_excel('/content/drive/MyDrive/Talent_Fair_HCK/aria_data.xlsx')

In [ ]:
data_ori.head()

In [ ]:
data_ori.tail()

In [ ]:
data = data_ori.copy()

### 2.1 ANALISIS KUALITAS DATA

Analisis kualitas data adalah proses untuk memeriksa dan mengevaluasi kualitas data untuk memastikan bahwa data yang digunakan untuk analisis atau pemodelan adalah akurat, konsisten, lengkap, dan relevan. Dalam analisis kualitas data, kita mencari tahu apakah terdapat data yang hilang, data yang tidak konsisten, atau data yang duplikat.

In [ ]:
# Checking Basic Information
data.info()

Berdasarkan dari informasi di atas, diketahui bahwa data ini terdiri dari 160 entri data dan terdiri dari total 10 kolom. Kolom tersebut memiliki tipe data float64 dan object. Selain itu, terlihat bahwa tidak terdapat nilai yang kosong dari setiap kolom berdasarkan data yang tersedia. Meskipun demikian, sebaiknya kita melakukan pemeriksaan lebih lanjut untuk memastikan ketiadaan nilai yang kosong pada data tersebut. Dengan melakukan pemeriksaan ini, kita dapat memastikan bahwa data yang digunakan untuk analisis lebih akurat dan valid.

In [ ]:
# Check Percentage Missing Values
data.isnull().mean().sort_values()

Setelah dilakukan pemeriksaan lebih lanjut, dapat disimpulkan bahwa memang tidak terdapat nilai yang kosong pada data ini. Oleh karena itu, tidak diperlukan adanya penanganan terhadap nilai yang kosong tersebut. Hasil ini memberikan kepastian bahwa data yang digunakan untuk analisis selanjutnya dapat dianggap akurat dan valid. Dengan demikian, data ini dapat dilanjutkan ke tahap analisis berikutnya tanpa harus mengkhawatirkan adanya nilai kosong yang dapat mempengaruhi hasil analisis.

In [ ]:
# Checking data duplicated
duplicates = data.duplicated().sum()
print('Duplicates:', duplicates)

Selain melakukan pemeriksaan terhadap nilai yang kosong pada data, kita juga melakukan pemeriksaan terhadap kemungkinan adanya nilai duplikat pada data. Dari hasil pemeriksaan, dapat disimpulkan bahwa tidak terdapat nilai duplikat pada data yang kita gunakan. Hal ini menunjukkan bahwa data tersebut bersifat unik dan berbeda satu sama lain. Dengan memastikan ketiadaan nilai duplikat pada data, kita dapat memastikan bahwa analisis yang dilakukan nantinya tidak terpengaruh oleh nilai duplikat yang tidak seharusnya ada dalam data tersebut. Dengan demikian, data yang digunakan untuk analisis selanjutnya dapat dianggap valid dan dapat diandalkan.

Selanjutnya, kita memeriksa apakah ada data yang tidak konsisten dalam setiap kolom dengan menghitung standar deviasi dari setiap kolom menggunakan fungsi apply() dan std(). Data yang tidak konsisten bisa menjadi indikasi adanya kesalahan dalam pengambilan data atau pengukuran.

In [ ]:
# Check for inconsistent data
inconsistent_data = data[['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8']].apply(lambda x: x.std(), axis=0)
print('Inconsistent Data:\n', inconsistent_data)

Dari hasil pemeriksaan yang dilakukan, ditemukan adanya inconsisten data pada kolom V1 hingga V8. Hal ini ditunjukkan oleh nilai yang sangat tinggi pada kolom V8, yaitu 335.035933. Nilai ini jauh lebih besar dibandingkan dengan nilai rata-rata kolom lainnya. Ketidaksesuaian ini dapat menyebabkan bias dalam analisis yang dilakukan karena adanya outlier yang mempengaruhi hasil keseluruhan. Oleh karena itu, sebaiknya kita melakukan pemeriksaan lebih lanjut apakah terdapat outlier atau tidak pada data tersebut.

Selanjutnya, untuk memastikan validitas analisis antara kedua sampel, perlu dilakukan pemeriksaan terhadap proporsi dari masing-masing sampel. Jika ditemukan bahwa proporsi kedua sampel tidak sama, maka perlu dilakukan penanganan terhadap kondisi imbalance tersebut sebelum dilakukan proses pemodelan machine learning. Hal ini dilakukan agar hasil analisis yang dihasilkan dapat lebih akurat dan dapat diandalkan dalam mengambil keputusan terkait pertumbuhan tanaman.

In [ ]:
data['sample_type'].value_counts()

Berdasarkan hasil dari pemeriksaan di atas bahwa proporsi dari masing-masing sampel tidak seimbang, hal ini dapat memengaruhi kualitas model machine learning yang akan dibangun. Untuk menangani ketidakseimbangan tersebut, dapat dilakukan teknik oversampling, yaitu menambahkan jumlah sampel pada sampel yang kurang hingga proporsi keduanya seimbang. Salah satu teknik oversampling yang umum digunakan adalah Synthetic Minority Oversampling Technique (SMOTE).

In [ ]:
# Pisahkan fitur dan label
X = data.drop('sample_type', axis=1)
y = data['sample_type']

# Inisialisasi SMOTE
sm = SMOTE(random_state=42)

# Resampling data
X_res, y_res = sm.fit_resample(X, y)

resampled = pd.concat([X_res.reset_index(drop=True), y_res.reset_index(drop=True)], axis=1)
resampled

In [ ]:
resampled['sample_type'].value_counts()

Hasil dari penerapan SMOTE adalah menghasilkan jumlah sampel yang sama antara kedua jenis sampel, yaitu 100 sampel untuk masing-masing jenis sampel. Hal ini dapat meminimalkan kemungkinan terjadinya overfitting pada model machine learning yang dibangun, serta memastikan bahwa model memiliki kemampuan yang seimbang dalam memprediksi target pada kedua jenis sampel. Dalam hal ini, variabel-variabel yang signifikan terhadap target pada kedua jenis sampel dapat dibandingkan dengan lebih akurat, karena tidak dipengaruhi oleh perbedaan jumlah sampel yang signifikan antara keduanya.

### 2.2 IDENTIFIKASI FAKTOR-FAKTOR

Identifikasi faktor-faktor yang memengaruhi ketersediaan nutrisi pada tanaman adalah proses analisis untuk menemukan hubungan antara variabel input atau faktor-faktor tertentu dengan ketersediaan nutrisi pada tanaman. Kita memeriksa hubungan antara variabel input atau faktor-faktor dengan ketersediaan nutrisi pada tanaman dengan menggunakan regresi linier.

Dalam melakukan analisis regresi, terdapat beberapa asumsi yang harus dipenuhi agar hasil analisis regresi dapat diandalkan. Beberapa asumsi tersebut antara lain normalitas data, homoskedastisitas, dan tidak adanya multikolinearitas antara variabel independent. Oleh karena itu, sebaiknya dilakukan juga uji asumsi regresi sebelum melakukan analisis regresi.

#### 2.2.1 NORMALITAS DATA

Untuk memeriksa normalitas data, kita akan menggunakan uji statistik seperti uji Shapiro-Wilk.

Shapiro-Wilk test adalah salah satu tes statistik yang dapat digunakan untuk menguji normalitas data. Hasil dari tes ini dapat dijelaskan sebagai berikut:

- H0: Data diuji berasal dari distribusi normal.
- H1: Data diuji tidak berasal dari distribusi normal.

Jika nilai p (p-value) dari Shapiro-Wilk test lebih besar dari tingkat signifikansi yang telah ditentukan, maka hipotesis nol diterima, dan data dianggap berasal dari distribusi normal. Sebaliknya, jika nilai p lebih kecil dari tingkat signifikansi, maka hipotesis alternatif diterima, dan data dianggap tidak berasal dari distribusi normal.

Umumnya, jika nilai p kurang dari 0,05 (tingkat signifikansi 5%), maka dapat dianggap bahwa data tidak berdistribusi normal. Namun, hasil dari Shapiro-Wilk test juga harus dilihat bersamaan dengan plot distribusi data, seperti histogram atau plot density, serta plot qq-plot, untuk memastikan apakah data benar-benar berdistribusi normal atau tidak.

QQ-plot (Quantile-Quantile plot) adalah grafik yang menunjukkan plot antara kuantil dari distribusi data dengan kuantil dari distribusi normal. Jika data normal, maka plot akan mengikuti garis diagonal. Sementara jika data tidak normal, plot akan memiliki pola yang berbeda.

In [ ]:
# Plot histogram
sns.histplot(resampled['target'], kde=True)

# Plot normal probability plot
stats.probplot(resampled['target'], dist="norm", plot=plt)

# Perform Shapiro-Wilk test
shapiro_test = stats.shapiro(resampled['target'])
print("Shapiro-Wilk test result:", shapiro_test)

Berdasarkan dari Grafik histogram di atas menunjukkan bahwa data memiliki pola yang simetris dan berbentuk kurva lonceng. Sementara itu, pada grafik QQ-plot, plot data mengikuti garis diagonal, menunjukkan bahwa data berdistribusi normal.

#### 2.2.2 HOMOSKEDASTISITAS

Homoskedastisitas adalah asumsi dalam analisis regresi yang menyatakan bahwa variansi residual (selisih antara nilai sebenarnya dan nilai yang diprediksi) dari model regresi harus sama konstan (homogen) di seluruh rentang nilai dari variabel independen. Dalam kondisi homoskedastisitas, sebaran nilai residual seharusnya merata di sepanjang rentang nilai variabel independen, sehingga tidak terlihat pola tertentu dalam plot residual.

Untuk memeriksa homoskedastisitas, kita akan menggunakan scatter plot. Jika pola yang dihasilkan menunjukkan pola funnel atau diamond, maka homoskedastisitas tidak terpenuhi. Jika pola yang dihasilkan menyebar merata dengan variansi yang relatif konstan, maka homoskedastisitas terpenuhi.

In [ ]:
# define dependent and independent variables
y = resampled['target']
X = resampled[['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8']]

import statsmodels.api as sm_api

# fit linear regression model
model = sm_api.OLS(y, sm_api.add_constant(X)).fit()

# check homoscedasticity
y_pred = model.predict(sm_api.add_constant(X))
residuals = y - y_pred

plt.scatter(y_pred, residuals)
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

Berdasarkan hasil dari scatter plot di atas, pola yang dihasilkan tidak menunjukkan pola funnel atau diamond dan menyebar merata dengan variansi yang relatif konstan, maka dapat disimpulkan bahwa homoskedastisitas pada data ini terpenuhi

#### 2.2.3 TIDAK ADANYA MULTIKOLINEARITAS

Untuk memeriksa multikolinearitas, kita akan menggunakan uji statistik seperti uji VIF (variance inflation factor).

In [ ]:
# Calculate VIF for each variable
vif = pd.DataFrame()
vif["variables"] = ['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8']
vif["VIF"] = [variance_inflation_factor(resampled[['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8']].values, i) for i in range(8)]
print(vif)

Hasil pemeriksaan multikolinearitas menunjukkan nilai Variance Inflation Factor (VIF) yang cukup tinggi untuk beberapa variabel, yaitu v3, v4, v5, dan v7, dengan nilai VIF masing-masing di atas 1000. Nilai VIF yang tinggi menunjukkan adanya korelasi yang tinggi antara variabel tersebut dengan variabel lain dalam model.

Korelasi yang tinggi antar variabel dapat menyebabkan masalah dalam analisis regresi, di mana hasil estimasi parameter regresi dapat menjadi tidak stabil atau tidak dapat diandalkan. Selain itu, interpretasi hasil analisis regresi juga dapat menjadi sulit karena adanya variasi yang tinggi dalam koefisien regresi.

Untuk mengatasi masalah multikolinearitas, salah satu langkah yang dapat dilakukan yaitu menghapus menghapus variabel yang memiliki nilai VIF yang sangat tinggi.

Dalam hal ini, dapat dipertimbangkan untuk mencoba menghapus variabel v4 dari model karena nilai VIF tertinggi kemudian akan dilakukan pemeriksaan kembali terhadap nilai VIF pada variabel lainnya.

In [ ]:
# Drop columns that have highest VIF values
resampled.drop(['v4'], axis = 1, inplace=True)
resampled

In [ ]:
# Calculate VIF for each variable
vif = pd.DataFrame()
vif["variables"] = ['v1', 'v2', 'v3', 'v5', 'v6', 'v7', 'v8']
vif["VIF"] = [variance_inflation_factor(resampled[['v1', 'v2', 'v3', 'v5', 'v6', 'v7', 'v8']].values, i) for i in range(7)]
print(vif)

Setelah dilakukan pemeriksaan ulang dan penghapusan variabel v4, ditemukan bahwa nilai VIF pada v7 sudah tidak terlalu tinggi seperti sebelumnya. Namun, terdapat variabel v3 yang memiliki nilai VIF yang cukup tinggi dibandingkan dengan variabel lainnya. Oleh karena itu, perlu dilakukan seleksi variabel dengan menghapus v3 dan melakukan pengecekan kembali terhadap variabel lainnya.

In [ ]:
# Drop columns that have highest VIF values
resampled.drop(['v3'], axis = 1, inplace=True)
resampled

In [ ]:
# Calculate VIF for each variable
vif = pd.DataFrame()
vif["variables"] = ['v1', 'v2', 'v5', 'v6', 'v7', 'v8']
vif["VIF"] = [variance_inflation_factor(resampled[['v1', 'v2', 'v5', 'v6', 'v7', 'v8']].values, i) for i in range(6)]
print(vif)

Dapat dilihat bahwa setelah dilakukan pemeriksaan ulang, variabel v5 masih memiliki nilai VIF yang cukup tinggi. Oleh karena itu, kita dapat mempertimbangkan untuk mencoba menghapus variabel v5 tersebut.

In [ ]:
# Drop columns that have highest VIF values
resampled.drop(['v5'], axis = 1, inplace=True)
resampled

In [ ]:
# Calculate VIF for each variable
vif = pd.DataFrame()
vif["variables"] = ['v1', 'v2', 'v6', 'v7', 'v8']
vif["VIF"] = [variance_inflation_factor(resampled[['v1', 'v2', 'v6', 'v7', 'v8']].values, i) for i in range(5)]
print(vif)

Berdasarkan hasil pemeriksaan ulang yang dilakukan, terlihat bahwa nilai VIF pada beberapa variabel telah menurun dan menjadi lebih rendah dibandingkan sebelumnya. Setelah dipertimbangkan, nilai VIF pada semua variabel di atas sudah berada pada tingkat yang dapat diterima, maka tidak perlu lagi dilakukan penghapusan variabel.

#### 2.2.4 ANALISIS REGRESI


Kita menggunakan formula smf.ols() dari paket statsmodels untuk membangun model regresi linier. Dalam formula regresi linier, Target adalah variabel dependent atau target yang ingin kita prediksi, V1, V2, V6, dan V8 adalah variabel independent atau faktor-faktor yang memengaruhi ketersediaan nutrisi pada tanaman, dan Sample_Type adalah variabel kategorikal yang mewakili dua jenis sampel yang diperoleh dari dua laboratorium yang berbeda.

In [ ]:
# Fit a linear regression model
model = smf.ols('target ~ v1 + v2 + v6 + v7 + v8 + sample_type', data=resampled).fit()

# Print the model summary
print(model.summary())

Hasil yang diberikan merupakan output dari regresi linear berganda (multiple linear regression) dengan menggunakan Ordinary Least Squares (OLS) sebagai metode estimasi. Model regresi ini mencoba untuk menjelaskan variasi pada variabel target (dependent variable) dengan menggunakan beberapa variabel prediktor (independent variable) yang tercantum dalam tabel.

Beberapa informasi yang dapat diambil dari hasil regresi ini antara lain:


*   R-squared (R2) atau koefisien determinasi sebesar 0.310. R2 adalah ukuran seberapa baik variabel independen dapat menjelaskan variasi pada variabel dependen. Semakin tinggi nilai R2, semakin baik variabel independen dalam menjelaskan variasi pada variabel dependen. Nilai R2 sebesar 0.310 menunjukkan bahwa variabel independen yang digunakan dalam model hanya dapat menjelaskan sekitar 31.0% variasi pada variabel dependen, sehingga masih ada sekitar 69.0% variasi lain yang belum dijelaskan oleh variabel independen.

*   Adj.R-squared (Adjusted R2) sebesar 0.288. Nilai ini adalah R2 yang disesuaikan untuk jumlah variabel independen yang digunakan dalam model. Semakin banyak variabel independen yang digunakan dalam model, semakin tinggi nilai Adj.R2, namun jika variabel independen yang ditambahkan tidak signifikan, nilai Adj.R2 malah dapat turun. Nilai Adj.R2 ini lebih rendah dari R2, yang menunjukkan bahwa penambahan variabel independen dalam model belum memberikan kontribusi signifikan terhadap penjelasan variasi pada variabel dependen.

*   F-statistic sebesar 14.44, dengan Prob (F-statistic) atau p-value sebesar 1.38e-13. F-statistic adalah ukuran kebermaknaan model regresi secara keseluruhan. Semakin tinggi nilai F-statistic, semakin baik model dalam menjelaskan variasi pada variabel dependen. Pada hasil di atas, nilai F-statistic cukup tinggi dengan p-value yang sangat rendah, sehingga model regresi dapat dianggap signifikan secara statistik.

*   Durbin-Watson (DW) sebesar 2.049. DW adalah ukuran autokorelasi pada residual (sisa) model regresi. Nilai DW berkisar antara 0 dan 4, dengan nilai 2 menunjukkan bahwa tidak ada autokorelasi yang signifikan pada residual model. Pada hasil di atas, nilai DW sebesar 2.049 menunjukkan bahwa tidak ada autokorelasi yang signifikan pada residual model.

*   Tabel koefisien menunjukkan nilai estimasi koefisien, standar error (std err), t-statistic, dan p-value untuk setiap variabel independen dalam model. Koefisien mengindikasikan arah dan besar pengaruh variabel independen terhadap variabel dependen. Nilai t-statistic dan p-value digunakan untuk menentukan signifikansi statistik dari koefisien. Jika nilai p-value kurang dari alpha (tingkat signifikansi yang ditentukan sebelumnya), maka koefisien dianggap signifikan secara statistik. Dalam hasil di atas, hanya variabel Intercept yang signifikan secara statistik dengan p-value 0.000

Berdasarkan hasil output tersebut, tidak dapat ditarik kesimpulan bahwa semua variabel independen memiliki pengaruh signifikan terhadap variabel target. Hal ini dapat dilihat dari nilai p-nilai (P>|t|) untuk setiap variabel independen, di mana semua nilai p-nilai melebihi tingkat signifikansi yang umumnya digunakan (biasanya 0,05).

Namun, perlu diperhatikan bahwa keberadaan pengaruh tidak signifikan secara statistik (terlihat dari nilai p-nilai) tidak berarti bahwa variabel independen tersebut sama sekali tidak memiliki pengaruh terhadap variabel target.

Oleh karena itu, diperlukan analisis yang lebih mendalam untuk menentukan variabel mana yang benar-benar memiliki pengaruh signifikan pada variabel target.

**Analisis terhadap dua tipe sampel**

Pada sesi ini akan dilakukan uji t-test antara dua sampel pada kolom target dari dataset yang tersedia.

Hasil dari ttest_ind adalah t-statistic dan p-value. t-statistic merupakan nilai t dari uji hipotesis t-test, sedangkan p-value merupakan nilai probabilitas yang menunjukkan seberapa signifikan perbedaan antara kedua sampel. Semakin kecil p-value, semakin signifikan perbedaan antara kedua sampel.

Setelah melakukan uji hipotesis, kita dapat mengevaluasi hasilnya berdasarkan nilai p-value. Jika nilai p-value lebih kecil dari tingkat signifikansi yang ditetapkan (biasanya 0.05), maka kita dapat menolak hipotesis nol dan menyimpulkan bahwa terdapat perbedaan yang signifikan antara kedua sampel. Sebaliknya, jika nilai p-value lebih besar dari tingkat signifikansi, maka kita gagal menolak hipotesis nol dan menyimpulkan bahwa tidak terdapat perbedaan yang signifikan antara kedua sampel.

In [ ]:
sample1 = resampled[resampled['sample_type'] == 'lab 1']['target']
sample2 = resampled[resampled['sample_type'] == 'lab 2']['target']

# perform t-test
t_stat, p_value = ttest_ind(sample1, sample2)

print("t-statistic:", t_stat)
print("p-value:", p_value)

Berdasarkan hasil pemeriksaan di atas menunjukkan bahwa t-statistik adalah -0,93 dan p-value adalah 0,35. Dalam kasus ini, p-value lebih besar dari 0,05, sehingga kita tidak dapat menolak hipotesis nol, yaitu tidak terdapat perbedaan signifikan antara kedua sampel. Oleh karena itu, dapat disimpulkan bahwa tidak terdapat perbedaan signifikan antara kedua sampel.

Meskipun hasil uji t tidak menunjukkan adanya perbedaan yang signifikan antara kedua sampel tersebut, namun masih dapat menggunakan variabel yang telah dipilih untuk melakukan prediksi menggunakan model machine learning. Karena tidak ada perbedaan signifikan antara sampel yang berbeda, maka kita bisa menganggap bahwa faktor-faktor yang mempengaruhi target pada kedua sampel tersebut relatif sama dan dapat dipertimbangkan secara serupa saat membangun model machine learning. Namun, perlu diingat bahwa tidak ada jaminan bahwa model yang dibangun dengan data tersebut akan memiliki kinerja yang sama baiknya pada kedua sampel. Hal ini karena meskipun faktor-faktor yang mempengaruhi target pada kedua sampel tersebut relatif sama, tetapi nilai dan kisaran dari masing-masing faktor pada kedua sampel tersebut bisa berbeda. Oleh karena itu, pada project ini tetap dipertimbangkan perbedaan karakteristik antara kedua sampel dalam membangun model machine learning.

In [ ]:
# Split data into two samples
lab1 = resampled[resampled['sample_type'] == 'lab 1']
lab2 = resampled[resampled['sample_type'] == 'lab 2']

In [ ]:
# Drop columns sample_type
lab1.drop(['sample_type'], axis = 1, inplace=True)
lab2.drop(['sample_type'], axis = 1, inplace=True)
lab2

## **III. EXPLORATORY DATA ANALYSIS**

## **IV. DATA PREPROCESSING**

### 4.1 SPLITTING THE DATA

#### 4.1.1 SAMPLE_TYPE:LAB 1

In [ ]:
# separating inference data
inference_lab1 = lab1.sample(10, random_state=32)
# reseting index
inference_lab1.reset_index(drop=True, inplace=True)
inference_lab1

In [ ]:
# separating inferential data from dataframe
lab1 = lab1.drop(inference_lab1.index)
# reseting index
lab1.reset_index(drop=True, inplace=True)
lab1

In [ ]:
# inference_lab1 = lab1.sample(10, random_state=15)
# lab1 = lab1.drop(inference_lab1.index)
# lab1.info()

In [ ]:
# define feature and target
X_lab1 = lab1.drop('target', axis=1)
y_lab1 = lab1.target

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_lab1, y_lab1, test_size=0.2, random_state=42)

for i in [X_train1, X_test1, y_train1, y_test1]:
    print(i.shape)

In [ ]:
# for EDA model creation, use dataframe stored in data_eda_lab1
data_eda_lab1 = pd.concat([X_train1.reset_index(drop=True), y_train1.reset_index(drop=True)], axis=1)
data_eda_lab1

In [ ]:
# Numerical Overview
data_eda_lab1.describe().T

In [ ]:
fig, ax = plt.subplots(ncols=3,nrows=2, figsize=[15,10])
ax = ax.flatten()

for idx, col in enumerate(data_eda_lab1):
    sns.histplot(data_eda_lab1[col], ax=ax[idx])
    ax[idx].set_title(f'{[idx]} skew: {data_eda_lab1[col].skew()}')

In [ ]:
def outlier_analysis(data_eda_lab1,col):
  skewness = data_eda_lab1[col].skew()
  if skewness>=-0.5 or skewness<=0.5:
    upper = data_eda_lab1[col].mean() + 3*data_eda_lab1[col].std()
    lower = data_eda_lab1[col].mean() - 3*data_eda_lab1[col].std()
  else:
    Q1 = data_eda_lab1[col].quantile(0.25)
    Q3 = data_eda_lab1[col].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q1 + (3*IQR)
    lower = Q3 - (3*IQR)
  
  no_outliers = data_eda_lab1[(data_eda_lab1[col]>=lower) & (data_eda_lab1[col]<=upper)]
  outliers = data_eda_lab1[(data_eda_lab1[col]<lower) | (data_eda_lab1[col]>upper)]
  print('percentage outlier from',i,':',outliers.shape[0]/data_eda_lab1.shape[0] * 100, '%')
  return outliers,no_outliers, upper, lower

for i in list(data_eda_lab1.columns):
  outlier_analysis(data_eda_lab1,i)

In [ ]:
wins = Winsorizer(capping_method='iqr', tail='both', fold=1.5, missing_values='ignore', variables=['v1', 'v6', 'v8'])

lab1_cleaned = wins.fit_transform(data_eda_lab1)

In [ ]:
# Compare before and after outlier handling
print('before handling: \n', data_eda_lab1.describe())

print('after handling: \n', lab1_cleaned.describe())

**SELECTION FEATURES NUMERIC**

In [ ]:
fig,ax = plt.subplots(figsize=[20,15])

corr = lab1_cleaned.corr()

ax = sns.heatmap(corr,annot=True, vmin=0, vmax=1)
plt.show()

**SCALING FEATURES NUMERIC**

In [ ]:
norm = 0
nonorm = 0
for col in lab1_cleaned[['v1', 'v2', 'v6', 'v7', 'v8']]:
  if lab1_cleaned[col].skew() >=-0.5 and lab1_cleaned[col].skew() <0.5:
    norm += 1
  else:
    nonorm +=1

if norm > nonorm:
  scaler = StandardScaler()
else:
  scaler = MinMaxScaler()
scaler

In [ ]:
# numerical scaling
num_col_scalling = ['v1', 'v2', 'v6', 'v7', 'v8']
scaler = MinMaxScaler()

num_scaled = pd.DataFrame(scaler.fit_transform(lab1_cleaned[num_col_scalling]))
num_scaled.columns = num_col_scalling

lab1_cleaned.drop(num_col_scalling, axis=1, inplace=True)
lab1_cleaned = pd.concat([lab1_cleaned.reset_index(drop=True), num_scaled], axis=1)

lab1_cleaned_num = lab1_cleaned[['v1', 'v2', 'v6', 'v7', 'v8', 'target']]
lab1_cleaned_num.head()

#### 4.1.2 SAMPLE_TYPE:LAB 2

In [ ]:
# separating inference data
inference_lab2 = lab2.sample(10, random_state=32)
# reseting index
inference_lab2.reset_index(drop=True, inplace=True)
inference_lab2

In [ ]:
# separating inferential data from dataframe
lab2 = lab2.drop(inference_lab2.index)
# reseting index
lab2.reset_index(drop=True, inplace=True)
lab2

In [ ]:
# inference_lab2 = lab2.sample(10, random_state=15)
# lab2 = lab2.drop(inference_lab2.index)
# lab2.info()

In [ ]:
# define feature and target
X_lab2 = lab2.drop('target', axis=1)
y_lab2 = lab2.target

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_lab2, y_lab2, test_size=0.2, random_state=42)

for i in [X_train2, X_test2, y_train2, y_test2]:
    print(i.shape)

In [ ]:
# for EDA model creation, use dataframe stored in data_eda_lab2
data_eda_lab2 = pd.concat([X_train2.reset_index(drop=True), y_train2.reset_index(drop=True)], axis=1)
data_eda_lab2

In [ ]:
# Numerical Overview
data_eda_lab2.describe().T

In [ ]:
fig, ax = plt.subplots(ncols=3,nrows=2, figsize=[15,10])
ax = ax.flatten()

for idx, col in enumerate(data_eda_lab2):
    sns.histplot(data_eda_lab2[col], ax=ax[idx])
    ax[idx].set_title(f'{[idx]} skew: {data_eda_lab2[col].skew()}')

In [ ]:
def outlier_analysis(data_eda_lab2,col):
  skewness = data_eda_lab2[col].skew()
  if skewness>=-0.5 or skewness<=0.5:
    upper = data_eda_lab2[col].mean() + 3*data_eda_lab2[col].std()
    lower = data_eda_lab2[col].mean() - 3*data_eda_lab2[col].std()
  else:
    Q1 = data_eda_lab2[col].quantile(0.25)
    Q3 = data_eda_lab2[col].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q1 + (1.5*IQR)
    lower = Q3 - (1.5*IQR)
  
  no_outliers = data_eda_lab2[(data_eda_lab2[col]>=lower) & (data_eda_lab2[col]<=upper)]
  outliers = data_eda_lab2[(data_eda_lab2[col]<lower) | (data_eda_lab2[col]>upper)]
  print('percentage outlier from',i,':',outliers.shape[0]/data_eda_lab2.shape[0] * 100, '%')
  return outliers,no_outliers, upper, lower

for i in list(data_eda_lab2.columns):
  outlier_analysis(data_eda_lab2,i)

**SELECTION FEATURES NUMERIC**

In [ ]:
fig,ax = plt.subplots(figsize=[20,15])

corr = data_eda_lab2.corr()

ax = sns.heatmap(corr,annot=True, vmin=0, vmax=1)
plt.show()

**SCALING FEATURES NUMERIC**

In [ ]:
norm = 0
nonorm = 0
for col in data_eda_lab2[['v1', 'v2', 'v6', 'v7', 'v8']]:
  if data_eda_lab2[col].skew() >=-0.5 and data_eda_lab2[col].skew() <0.5:
    norm += 1
  else:
    nonorm +=1

if norm > nonorm:
  scaler = StandardScaler()
else:
  scaler = MinMaxScaler()
scaler

In [ ]:
# numerical scaling
num_col_scalling = ['v1', 'v2', 'v6', 'v7', 'v8']
scaler = StandardScaler()

num_scaled = pd.DataFrame(scaler.fit_transform(data_eda_lab2[num_col_scalling]))
num_scaled.columns = num_col_scalling

data_eda_lab2.drop(num_col_scalling, axis=1, inplace=True)
data_eda_lab2 = pd.concat([data_eda_lab2.reset_index(drop=True), num_scaled], axis=1)

data_eda_lab2_num = data_eda_lab2[['v1', 'v2', 'v6', 'v7', 'v8', 'target']]
data_eda_lab2_num.head()

### 4.2 PREPROCESS

#### 4.2.1 TESTING:LAB 1

In [ ]:
testing_lab1 = pd.concat([X_test1.reset_index(drop=True), y_test1.reset_index(drop=True)], axis=1)
testing_lab1.head()

In [ ]:
# capping outlier
testing_lab1 = wins.transform(testing_lab1)
testing_lab1.head()

In [ ]:
# numerical scalling
num_col = ['v1', 'v2', 'v6', 'v7', 'v8']

testing_lab1[num_col] = scaler.transform(testing_lab1[num_col])
testing_lab1.head()

In [ ]:
X_train1 = lab1_cleaned_num.drop('target',axis=1).copy()
y_train1 = lab1_cleaned_num['target']

X_test1 = testing_lab1.drop('target',axis=1).copy()
y_test1 = testing_lab1['target']

In [ ]:
X_train1

In [ ]:
X_test1

#### 4.2.2 TESTING:LAB 2

In [ ]:
testing_lab2 = pd.concat([X_test2.reset_index(drop=True), y_test2.reset_index(drop=True)], axis=1)
testing_lab2.head()

In [ ]:
# numerical scalling
num_col = ['v1', 'v2', 'v6', 'v7', 'v8']

testing_lab2[num_col] = scaler.transform(testing_lab2[num_col])
testing_lab2.head()

In [ ]:
X_train2 = data_eda_lab2_num.drop('target',axis=1).copy()
y_train2 = data_eda_lab2_num['target']

X_test2 = testing_lab2.drop('target',axis=1).copy()
y_test2 = testing_lab2['target']

In [ ]:
X_train2

In [ ]:
X_test2

## V. MODEL TRAINING

#### 5.1.1 SAMPLE_TYPE:LAB 1

In [ ]:
auto_regressor1 = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=300,
    per_run_time_limit=60,
)
auto_regressor1.fit(X_train1, y_train1)

In [ ]:
print(auto_regressor1.leaderboard())

#### 5.1.2 SAMPLE_TYPE:LAB 2

In [ ]:
auto_regressor2 = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=300,
    per_run_time_limit=60,
)
auto_regressor2.fit(X_train2, y_train2)

In [ ]:
print(auto_regressor2.leaderboard())

## VI. MODEL EVALUATION


#### 6.1.1 SAMPLE_TYPE:LAB 1

In [ ]:
train_predict1 = auto_regressor1.predict(X_train1)
test_predict1 = auto_regressor1.predict(X_test1)

print('------- TRAIN EVALUATION -------')
print('MAE :', mae(y_train1, train_predict1))
print('RMSE:', np.sqrt(mse(y_train1, train_predict1)))
print(' ')
print('------- TEST EVALUATION -------')
print('MAE :', mae(y_test1, test_predict1))
print('RMSE:', np.sqrt(mse(y_test1, test_predict1)))

#### 6.1.2 SAMPLE_TYPE:LAB 2

In [ ]:
train_predict2 = auto_regressor2.predict(X_train2)
test_predict2 = auto_regressor2.predict(X_test2)

print('------- TRAIN EVALUATION -------')
print('MAE :', mae(y_train2, train_predict2))
print('RMSE:', np.sqrt(mse(y_train2, train_predict2)))
print(' ')
print('------- TEST EVALUATION -------')
print('MAE :', mae(y_test2, test_predict2))
print('RMSE:', np.sqrt(mse(y_test2, test_predict2)))

## VII. MODEL INFERENCE

#### 7.1.1 SAMPLE_TYPE:LAB 1

In [ ]:
inference_lab1

#### 7.1.2 SAMPLE_TYPE:LAB 2